In [ ]:
# Ch11-1-plink [Updated]

In [ ]:
# Import libraries
import os

In [ ]:
# Install Plink
#. Download the appropriate binary for your system from here:
#. https://www.cog-genomics.org/plink/2.0/
# Move the file from your Downloads directory to your Ch11 working directory 
# unzip the file
# Test by running: 
# ./plink2
# You will most likely get a message saying you cannot trust this file.  Go into your Mac Settings ->
# Open System Preferences → Security & Privacy:
# Click on the General tab.
# Look for a message saying that "plink2 was blocked because it is from an unidentified developer."
# Click Allow Anyway
# you should now be able to run ./plink2 
# You may need to click Allow All again and provide the administrator password for your Mac

In [ ]:
# Download the data
# First go here and download the 2 files:
# https://ftp.ncbi.nlm.nih.gov/hapmap/genotypes/hapmap3_r3/plink_format/
# move them to your Ch11/data working directory
# Unzip the files:
! gunzip data/hapmap3_r3_b36_fwd.consensus.qc.poly.map.gz 
! gunzip data/hapmap3_r3_b36_fwd.consensus.qc.poly.ped.gz 
# Get the relationships file
! wget https://ftp.ncbi.nlm.nih.gov/hapmap/genotypes/hapmap3_r3/relationships_w_pops_041510.txt
! mv relationships_w_pops_041510.txt data/

In [ ]:
# 1 - get metadata for samples
from collections import defaultdict 
f = open('data/relationships_w_pops_041510.txt') 
pop_ind = defaultdict(list) 
f.readline() # header 
offspring = [] 
for l in f: 
    toks = l.rstrip().split('\t') 
    fam_id = toks[0] 
    ind_id = toks[1] 
    mom = toks[2] 
    dad = toks[3] 
    if mom != '0' or dad != '0': 
        offspring.append((fam_id, ind_id)) 
    pop = toks[-1] 
pop_ind[pop].append((fam_id, ind_id)) 
f.close() 

In [ ]:
# 2 - subsample the data
#. Note - replace the path below with your path to plink2
! ~/work/CookBook/Ch11/plink2 --pedmap data/hapmap3_r3_b36_fwd.consensus.qc.poly --out hapmap10 --thin 0.1 --geno 0.1 --export ped 
! ~/work/CookBook/Ch11/plink2 --pedmap data/hapmap3_r3_b36_fwd.consensus.qc.poly --out hapmap1 --thin 0.01 --geno 0.1 --export ped 

In [ ]:
# 3. Generate subsets with just the autosomes
def get_non_auto_SNPs(map_file, exclude_file): 
    f = open(map_file) 
    w = open(exclude_file, 'w') 
    for l in f: 
        toks = l.rstrip().split('\t') 
        try: 
            chrom = int(toks[0]) 
        except ValueError: 
            rs = toks[1] 
            w.write('%s\n' % rs) 
    w.close() 
get_non_auto_SNPs('hapmap1.map', 'exclude1.txt') 
get_non_auto_SNPs('hapmap10.map', 'exclude10.txt') 
os.system('~/work/CookBook/Ch11/plink2 --pedmap hapmap1 --out hapmap1_auto --exclude exclude1.txt --export ped') 
os.system('~/work/CookBook/Ch11/plink2 --pedmap hapmap10 --out hapmap10_auto --exclude exclude10.txt --export ped') 

In [ ]:
# 4. Function to generate a list of SNPs belonging to autosomes

In [ ]:
# 6. Datasets without offspring
os.system('~/work/CookBook/Ch11/plink2 --pedmap hapmap10_auto --filter-founders --out hapmap10_auto_noofs --export ped') 

In [ ]:
# 7. Generate LD-pruned dataset
os.system('~/work/CookBook/Ch11/plink2 --pedmap hapmap10_auto_noofs --indep-pairwise 50 10 0.1 --out keep --export ped') 
#os.system('~/work/CookBook/Ch11/plink2 --pedmap hapmap10_auto_noofs --extract keep.prune.in --recode --out hapmap10_auto_noofs_ld --export ped')
# Remove --recode from the original commmand to remove an error: 
os.system('~/work/CookBook/Ch11/plink2 --pedmap hapmap10_auto_noofs --extract keep.prune.in --out hapmap10_auto_noofs_ld --export ped')

In [ ]:
# 8.  Recode cases in different formats
# os.system('~/work/CookBook/Ch11/plink2 --file hapmap10_auto_noofs_ld --recode12 tab --out hapmap10_auto_noofs_ld_12 --export ped 12') 
#. Note - fixed above original command to not use --file
os.system('~/work/CookBook/Ch11/plink2 --pedmap hapmap10_auto_noofs_ld --export ped --out hapmap10_auto_noofs_ld_12')
# os.system('~/work/CookBook/Ch11/plink2 --make-bed --file hapmap10_auto_noofs_ld --out hapmap10_auto_noofs_ld') 
# Note - fixed above original command to not use --file
os.system('~/work/CookBook/Ch11/plink2 --pedmap hapmap10_auto_noofs_ld --make-bed --out hapmap10_auto_noofs_ld')

In [ ]:
# 9. Extract a single chromosome for analysis
os.system('~/work/CookBook/Ch11/plink2 --pedmap hapmap10_auto_noofs --chr 2 --out hapmap10_auto_noofs_2 --export ped') 

In [ ]:
## End of Notebook ##